In [30]:
import numpy as np 
import pandas as pd

In [31]:
col_names = ['Age','Workclass','FNLWGT','Education','EducationNum','MaritalStatus','Occupation','Relationship','Race','Sex','CapitalGain','CapitalLoss','HoursPerWeek','NativeCountry','Income'
]
data = pd.read_csv('adult.data',skiprows=1,names=col_names)
data = data.head(5000)

In [32]:
class Node():
    def __init__(self,feature_index=None,threshold=None,left=None,right=None,info_gain=None,value=None):
        
        # decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        # leaf node
        self.value = value

In [1]:
class DecisionTreeClassifier():
    def __init__(self,min_samples_split=2,max_depth=2):
        
        self.root = None
        self.min_sample_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self,dataset,current_depth = 0):
        
        X,Y = dataset[:,:-1],dataset[:,-1]
        num_samples,num_features =np.shape(X)
        
        # stopping conditions
        if num_samples >= self.min_sample_split and current_depth <= self.max_depth:
            # neither of the stopping condition is met...find best split
            best_split = self.get_best_split(dataset,num_samples,num_features)
            # check information gain
            if best_split['info_gain'] > 0:
                # recur left
                left_subtree = self.build_tree(best_split['dataset_left'],current_depth +1)
                # recur right
                right_subtree = self.build_tree(best_split['dataset_right'],current_depth +1)
                # return decision node
                return Node(best_split['feature_index'],best_split['threshold'],
                            left_subtree,right_subtree,best_split['info_gain'])

        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)    
        # return leaf node
        return Node(value = leaf_value)
    
    def get_best_split(self,dataset,num_samples,num_features):
        
        best_split = {}
        max_info_gain = -float('inf')
        
        for feature_index in range(num_features):
            feature_values = dataset[:,feature_index]
            possible_threshold = np.unique(feature_values)
            for threshold in possible_threshold:
                # get current split
                dataset_left,dataset_right = self.split(dataset,feature_index,threshold)
                # check if children are not null
                if len(dataset_right) > 0 and len(dataset_left) > 0:
                    y, left_y, right_y = dataset[:,-1], dataset_left[:,-1], dataset_right[:,-1]
                    # compute information gain 
                    curr_info_gain = self.information_gain(y,left_y,right_y,'gini')
                    #update information gain if needed
                    if curr_info_gain > max_info_gain:
                        best_split['feature_index'] = feature_index
                        best_split['threshold'] = threshold
                        best_split['dataset_left'] = dataset_left
                        best_split['dataset_right'] = dataset_right
                        best_split['info_gain'] = curr_info_gain
                        max_info_gain = curr_info_gain
        return best_split
    
    def split(self,dataset,feature_index, threshold):
        
        dataset_left = np.array([row for row in dataset if row[feature_index] <= threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index] > threshold])
        
        return dataset_left,dataset_right
    
    def information_gain(self,parent,l_child,r_child, mode = 'entropy'):
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        
        if mode == 'gini':
            gain = self.gini_index(parent) - (weight_l * self.gini_index(l_child) + weight_r * self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l * self.entropy(l_child) + weight_r * self.entropy(r_child))
        return gain 
    
    def entropy(self, y):
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y==cls]) /len(y)
            entropy += -p_cls * np.log2(p_cls)
            return entropy
        
    def gini_index(self, y):
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y==cls]) /len(y)
            gini += p_cls ** 2 
            return 1 - gini
    
    def calculate_leaf_value(self, Y):
        
        Y = list(Y)
        return max(Y,key=Y.count)
    
    def print_tree(self, tree = None, indent = " "):
        
        if not tree:
            tree = self.root
        
        if tree.value is not None:
            print(tree.value)
        else:
            print(f'X_{str(tree.feature_index)} <= {tree.threshold} ? {tree.info_gain}')
            print('%sleft:'%(indent),end="")
            self.print_tree(tree.left,indent + indent)
            print('%right:'%(indent),end="")
            self.print_tree(tree.right,indent + indent)
            
    
    def fit(self, X, Y):
        '''function to train the tree'''
        dataset = np.concatenate((X,Y),axis=1)
        self.root = self.build_tree(dataset)
        
    def predict_regr(self, X):
        '''function to predict_regr a new dataset'''
        predictions = [self.make_regression(x,self.root) for x in X]
        return predictions
    
    def make_regression(self,x,tree):
        '''function to make a single prediction'''
        if tree.value != None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self.make_regression(x,tree.left)
        else:
            return self.make_regression(x,tree.right)

In [34]:
X = data.iloc[:,:-1].values
Y = data.iloc[:,-1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=.2,random_state=7)

In [35]:
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train,Y_train)

In [36]:
classifier.print_tree()

X_7 <=  Husband ? 0.028585775000652458
 left:X_10 <= 10566 ? 0.04983032285735778
  left:X_4 <= 11 ? 0.02452353624540149
    left:X_10 <= 5013 ? 0.016786721928056414
        left: <=50K
'        'ight: >50K
'    'ight:X_11 <= 1740 ? 0.12528776236201278
        left: >50K
'        'ight: >50K
'  'ight: >50K
' 'ight:X_7 <=  Unmarried ? 0.015615382717628273
  left:X_10 <= 6849 ? 0.011599726261888474
    left:X_4 <= 12 ? 0.0027378032112795325
        left: <=50K
'        'ight: <=50K
'    'ight:X_0 <= 18 ? 0.9963269054178145
        left: <=50K
'        'ight: >50K
'  'ight:X_11 <= 1740 ? 0.0782844159482089
    left:X_1 <=  Self-emp-not-inc ? 0.04979102851181727
        left: <=50K
'        'ight: >50K
'    'ight: >50K


In [37]:
Y_pred = classifier.predict_regr(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,Y_pred)

0.821